<table align="left">
    <tr>
        <td style="vertical-align: middle; padding-left: 0px; padding-right: 0px;">
            <a href="https://creativecommons.org/licenses/by/4.0/">
                <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 5px; padding-right: 0px;">
            <a href="https://opensource.org/licenses/MIT">
                <img src="https://img.shields.io/badge/License-MIT-green.svg" />
            </a>
        </td>
        <td style="vertical-align: middle; padding-left: 15px;">
            &copy; Guillaume Rongier
        </td>
    </tr>
</table>

# Building a simple numerical model of a delta

*Based on [Building a simple delta numerical model](https://www.andrewjmoodie.com/blog/2016/11-01-2016-building-a-simple-delta-numerical-model-part-i/) by Andrew J. Moodie.*

In this notebook, we will build a simple numerical model for a delta piece by piece before putting all the pieces together.

### Imports

In [ ]:
# To manipulate arrays and matrices
import numpy as np
# To create plots and visualizations
import matplotlib.pyplot as plt
# To add widgets for interactive visualizations
import ipywidgets as widgets
# To create animated visualizations
import matplotlib.animation as animation
# To display animated visualizations
from IPython.display import HTML

### Setting

In [ ]:
%matplotlib widget

## 1. Basic components

### 1.1. Water flow

This section is based on these two blog posts:
 * https://www.andrewjmoodie.com/blog/2016/11-01-2016-building-a-simple-delta-numerical-model-part-i/
 * https://www.andrewjmoodie.com/blog/2016/11-08-2016-building-a-simple-delta-numerical-model-part-ii/

Here we consider a [gradually-varied flow](https://en.wikipedia.org/wiki/Open-channel_flow) under steady state conditions in an open channel of rectangular cross section and constant width. In such setting, we can get the one-dimensional water surface elevation using the [standard step method](https://en.wikipedia.org/wiki/Standard_step_method), in which we need to solve the gradually-varied-flow equation:
$$
\frac{\partial H}{\partial x} = \frac{S - C_f Fr^2}{1 - Fr^2}
$$
With:
$$
\begin{align}
Fr^2 & = \frac{q_w^2}{g H^3} \\[0.6em]
q_w & = Q_w / B \\
\end{align}
$$
Where:
 * $H$ is the flow depth [L].
 * $x$ is the streamwise coordinate [L].
 * $S$ is the slope of the channel bed [L/L].
 * $C_f$ is the coefficient of friction [dimensionless].
 * $Fr$ is the [Froude number](https://en.wikipedia.org/wiki/Froude_number) [dimensionless].
 * $g$ is the gravitational acceleration constant [L/T$^2$].
 * $q_w$ is the width-averaged water discharge [L$^2$/T].
 * $B$ is the channel width [L].
 * $Q_w$ is the water discharge such as $Q_w = U A$ [L$^3$/T].
 * $U$ is the flow velocity [L/T].
 * $A$ is the channel cross-sectional area given by $A = B H$ [L$^2$].

To do so, we will start downstream and then move upstream (backwater calculation) using the [predictor-corrector method](https://en.wikipedia.org/wiki/Predictor%E2%80%93corrector_method).

Let's define two functions to do that:

In [ ]:
def gradually_varied_flow(water_discharge,
                          flow_depth,
                          channel_width,
                          bed_slope,
                          friction_coef,
                          g=9.81):
    """
    Computes the change of flow depth along the streamwise direction for a
    gradually-varied flow in an open channel.

    Parameters
    ----------
    water_discharge : float
        The water discharge into the channel [L3/T].
    flow_depth : float
        The current flow depth [L].
    channel_width : float
        The width of the channel [L].
    bed_slope : float
        The slope of the channel bed [dimensionless].
    friction_coef : float
        The coefficient of friction [dimensionless].
    g : float, default=9.81
        The gravitational acceleration constant [L/T2, default m/s2].

    Returns
    -------
    flow_depth_change : float or array-like
        The change of flow depth [L].
    """
    width_discharge = water_discharge/channel_width
    sq_froude_nb = width_discharge**2 / (g * flow_depth**3)

    return (bed_slope - friction_coef*sq_froude_nb)/(1 - sq_froude_nb)


def solve_backwater_profile(topography,
                            spacing,
                            channel_width,
                            friction_coef,
                            water_discharge,
                            base_level,
                            g=9.81):
    """
    Solves the gradually-varied-flow equation using the predictor-corrector
    method starting downstream then moving upstream.

    Parameters
    ----------
    topography : array-like
        The topopgraphy along the streamwise coordinate in a one-dimensional
        array [L].
    spacing : float
        The distance between the nodes of `topopgraphy` [L].
    channel_width : float
        The width of the channel [L].
    friction_coef : float
        The coefficient of friction [dimensionless].
    water_discharge : float
        The water discharge into the channel [L3/T].
    base_level : float
        The elevation of the water surface at the end of `topopgraphy`, on
        the downstream side.
    g : float, default=9.81
        The gravitational acceleration constant [L/T2, default m/s2].

    Returns
    -------
    flow_depth_change : float or array-like
        The change of flow depth [L].
    """
    bed_slope = -np.gradient(topography, spacing)

    flow_depth = np.empty_like(topography)
    flow_depth[-1] = base_level - topography[-1]
    for i in range(flow_depth.shape[0] - 1, 0, -1):
        flow_depth_change_p = gradually_varied_flow(water_discharge,
                                                    flow_depth[i],
                                                    channel_width,
                                                    bed_slope[i],
                                                    friction_coef,
                                                    g=g)
        flow_depth_change_c = gradually_varied_flow(water_discharge,
                                                    flow_depth[i] - spacing*flow_depth_change_p,
                                                    channel_width,
                                                    bed_slope[i - 1],
                                                    friction_coef,
                                                    g=g)
        flow_depth[i - 1] = flow_depth[i] - spacing*(flow_depth_change_p + flow_depth_change_c)/2

    return flow_depth

And some parameter values:

In [ ]:
# Length of the domain
extent = 1200e3 # m
# Number of nodes
shape = 400 # cell
# Size of each cell
spacing = extent/shape # m

# Elevation at x = 0 to initialize the topography
initial_elevation = 63 # m
# Slope of the initial topography
initial_slope = 7e-5 # m/m

# Fixed base level
base_level = 0 # m
# Constant channel width
channel_width = 1100 # m
# Friction coefficient
friction_coef = 0.0047

We can now compute the topography:

In [ ]:
# Coordinates of each node along the x axis
x = np.linspace(0, extent, shape + 1)
# Channel bed
initial_topography = np.linspace(initial_elevation,
                                 initial_elevation - initial_slope*shape*spacing,
                                 shape + 1)

And compute the flow depth for different values of water discharge:

In [ ]:
flow_depth_9000 = solve_backwater_profile(initial_topography,
                                          spacing,
                                          channel_width,
                                          friction_coef,
                                          9000, # m3/s
                                          base_level)
flow_depth_27000 = solve_backwater_profile(initial_topography,
                                           spacing,
                                           channel_width,
                                           friction_coef,
                                           27000, # m3/s
                                           base_level)
flow_depth_50000 = solve_backwater_profile(initial_topography,
                                           spacing,
                                           channel_width,
                                           friction_coef,
                                           50000, # m3/s
                                           base_level)

We can then plot the water surfaces corresponding to each water depth:

In [ ]:
fig, ax = plt.subplots()
fig.canvas.header_visible = False
ax.plot(x/1000, initial_topography + flow_depth_9000,
        c='tab:blue', label='Water surface, $Q_w = 9000~m^3/s$')
ax.plot(x/1000, initial_topography + flow_depth_27000,
        c='tab:blue', ls='dashed', label='Water surface, $Q_w = 27000~m^3/s$')
ax.plot(x/1000, initial_topography + flow_depth_50000,
        c='tab:blue', ls='dotted', label='Water surface, $Q_w = 50000~m^3/s$')
ax.plot(x/1000., initial_topography, c='black', label='Channel bed')
ax.set(xlim=(0, extent/1000), ylim=(-40, 100),
       xlabel='Distance downstream (km)', ylabel='Elevation (m)')
ax.legend();

Or even better define an interactive plot to look at the impact of the discharge on the water surface:

In [ ]:
fig, ax = plt.subplots()

args = {'continuous_update': False, 'layout': {'width': '450px'}, 'style': {'description_width': '150px'}}
wdg_discharge = widgets.IntSlider(value=9000, min=500, max=75000, step=500,
                                  description=u'Water discharge (m\u00B3/s)', **args)

def update(discharge):
    flow_depth = solve_backwater_profile(initial_topography, spacing,
                                         channel_width, friction_coef,
                                         discharge, base_level)
    line_depth.set_ydata(initial_topography + flow_depth)

fig.canvas.header_visible = False
line_depth, = ax.plot(x/1000, initial_topography + flow_depth_9000,
                      c='tab:blue', label='Water surface')
ax.plot(x/1000, initial_topography, c='black', label='Channel bed')
ax.set(xlim=(0, extent/1000), ylim=(-40, 100),
       xlabel='Distance downstream (km)', ylabel='Elevation (m)')
ax.legend()

# Run the interactive plot.
out = widgets.interactive_output(update, {'discharge': wdg_discharge})
widgets.HBox([wdg_discharge, out])

### 1.2. Sediment transport

This section is based on this blog post: https://www.andrewjmoodie.com/blog/2016/11-15-2016-building-a-simple-delta-numerical-model-part-iii/

Now that we have the flow depth, we can compute sediment transport based on the flow velocity using the Engelund-Hansen formula:
$$
q_s = \beta \sqrt{R g D} D \frac{0.05}{C_f} \left( \frac{\tau}{\rho R g D} \right)^{2.5}
$$
With:
$$
\begin{align}
\tau & = \rho C_f U^2 \\[0.6em]
U & = \frac{Q_w}{H B} \\
\end{align}
$$
Where:
 * $q_s$ is the sediment transport per unit width [L$^2$/T].
 * $\beta$ is an empirical adjustment coefficient [dimensionless].
 * $R$ is the submerged specific gravity of the bed sediment [dimensionless].
 * $g$ is the gravitational acceleration constant [L/T$^2$].
 * $D$ is the grain size [L].
 * $C_f$ is the coefficient of friction [dimensionless].
 * $\rho$ is the fluid density [M/L$^3$]
 * $\tau$ is the fluid shear stress [M$\cdot$L$^{-1}$$\cdot$T$^{-2}$].
 * $U$ is the depth-averaged flow velocity [L/T].
 * $Q_w$ is the water discharge [L$^3$/T].
 * $H$ is the flow depth [L].
 * $B$ is the channel width [L].

Let's define a function to compute that formula:

In [ ]:
def engelund_hansen_formula(grain_size,
                            flow_velocity,
                            fluid_density,
                            friction_coef,
                            submerged_specific_gravity,
                            beta,
                            g=9.81):
    """
    Computes sediment transport using the Engelund-Hansen formula.

    Parameters
    ----------
    grain_size : float
        The grain size [L].
    flow_velocity : float or array-like
        The flow velocity in a float or a one-dimensional array [L/T].
    fluid_density : float
        The fluid density [M/L]3].
    friction_coef : float
        The coefficient of friction [dimensionless].
    submerged_specific_gravity : float
        The submerged specific gravity [dimensionless].
    beta : float
        An empirical adjustment coefficient [dimensionless].
    g : float, default=9.81
        The gravitational acceleration constant [L/T2, default m/s2].

    Returns
    -------
    width_sediment_flux : float or array-like
        The sediment transport per unit width [L2/T].
    """
    fluid_shear_stress = fluid_density * friction_coef * flow_velocity**2
    factor = beta * np.sqrt(submerged_specific_gravity*g*grain_size) * grain_size * 0.05 / friction_coef
    width_sediment_flux = factor * (fluid_shear_stress/(fluid_density*submerged_specific_gravity*g*grain_size))**2.5

    return width_sediment_flux

And some parameter values:

In [ ]:
# Water discharge
water_discharge = 10000 # m3/s
# Water density
fluid_density = 1000 # kg/m3

# Grain size
grain_size = 0.0003 # m
# Submerged specific gravity
submerged_specific_gravity = 1.65
# Empirical adjustment coefficient
beta = 0.64

Let's define a range of flow velocity values to look at the impact of $\beta$ on sediment transport:

In [ ]:
flow_velocity = np.linspace(0., 3., 1000) # m/s

And compute the sediment transport for two different values of $\beta$:

In [ ]:
sediment_transport_064 = engelund_hansen_formula(grain_size,
                                                 flow_velocity,
                                                 fluid_density,
                                                 friction_coef,
                                                 submerged_specific_gravity,
                                                 0.64)
sediment_transport_1 = engelund_hansen_formula(grain_size,
                                               flow_velocity,
                                               fluid_density,
                                               friction_coef,
                                               submerged_specific_gravity,
                                               1)

We can then plot the sediment transport corresponding to each $\beta$:

In [ ]:
fig, ax = plt.subplots()
fig.canvas.header_visible = False
ax.plot(flow_velocity, sediment_transport_1, c='black', label=r'$\beta = 1$')
ax.plot(flow_velocity, sediment_transport_064, c='black', ls='dashed', label=r'$\beta = 0.64$')
ax.set(xlim=(0, 3), ylim=(0, 0.05),
       xlabel='Velocity (m/s)', ylabel='Transport (m$^2$/s)')
ax.legend();

Let's combine this with the previous section:

In [ ]:
flow_depth = solve_backwater_profile(initial_topography,
                                     spacing,
                                     channel_width,
                                     friction_coef,
                                     water_discharge,
                                     base_level)
flow_velocity = water_discharge/(flow_depth*channel_width)
sediment_transport = engelund_hansen_formula(grain_size,
                                             flow_velocity,
                                             fluid_density,
                                             friction_coef,
                                             submerged_specific_gravity,
                                             beta)

And plot the result:

In [ ]:
fig, axs = plt.subplots(nrows=2, sharex=True, layout='constrained')
fig.canvas.header_visible = False

axs[0].plot(x/1000, initial_topography + flow_depth,
            c='tab:blue', label='Water surface')
axs[0].plot(x/1000, initial_topography, c='black', label='Channel bed')
axs[0].set(xlim=(0, extent/1000), ylim=(-40, 80), ylabel='Elevation (m)')
axs[0].legend()

axs[1].plot(x/1000, flow_velocity, c='tab:red')
axs[1].set(xlim=(0, extent/1000), ylim=(0, 2), xlabel='Distance downstream (km)')
axs[1].set_ylabel('Velocity (m/s)', color='tab:red')
axs[1].tick_params(axis='y', color='tab:red', labelcolor='tab:red')
ax2 = axs[1].twinx()
ax2.plot(x/1000, sediment_transport, c='tab:green')
ax2.set(xlim=(0, extent/1000), ylim=(0, 4e-4))
ax2.set_ylabel('Transport (m$^2$/s)', color='tab:green')
ax2.tick_params(axis='y', color='tab:green', labelcolor='tab:green')
ax2.spines['right'].set_color('tab:green')
ax2.spines['left'].set_color('tab:red');

### 1.3. Erosion and deposition

This section is based on this blog post: https://www.andrewjmoodie.com/blog/2017/01-29-2017-building-a-simple-delta-numerical-model-part-iv/

Now that we have the sediment transport, we can compute erosion and deposition using the [Exner equation](https://en.wikipedia.org/wiki/Exner_equation):
$$
\frac{\partial \eta}{\partial t} = - \frac{1}{1 - \lambda_p} \frac{\partial q_s}{\partial x}
$$
Where:
 * $\eta$ is the bed elevation [L].
 * $t$ is time [T].
 * $\lambda_p$ is the channel-bed porosity [dimensionless].
 * $q_s$ is the sediment transport per unit width [L$^2$/T].
 * $x$ is the streamwise coordinate [L].

Let's define a function to compute that equation:

In [ ]:
def exner_equation(sediment_transport,
                   spacing,
                   time_step,
                   intermittency_factor,
                   bed_porosity,
                   upstream_sediment_transport,
                   winding_coef):
    """
    Computes the change of elevation over time based on Exner equation.

    Parameters
    ----------
    sediment_transport : float or array-like
        The sediment transport in a float or a one-dimensional array [L2/T].
    spacing : float
        The distance between the nodes of `sediment_transport` [L].
    time_step : float
        Period of time over which the change of elevation took place [T].
    intermittency_factor : float
        Fraction of the year that the river is experiencing significant
        morphodynamic activity [dimensionless].
    bed_porosity : float
        The bed porosity [dimensionless].
    upstream_sediment_transport : float
        The sediment transport coming from the upstream boundary at the start
        of `sediment_transport` [L2/T].
    winding_coef : float
        The winding coefficient for solving the equation [dimensionless].

    Returns
    -------
    elevation_change : float or array-like
        The change of elevation over time [L/T].
    """
    elevation_change = np.empty_like(sediment_transport)

    elevation_change[0] = (winding_coef*(sediment_transport[0] - upstream_sediment_transport)/spacing +
                           (1 - winding_coef)*(sediment_transport[1] - sediment_transport[0])/spacing)
    elevation_change[1:-1] = (winding_coef*(sediment_transport[1:-1] - sediment_transport[:-2])/spacing +
                              (1 - winding_coef)*(sediment_transport[2:] - sediment_transport[1:-1])/spacing)
    elevation_change[-1] = (sediment_transport[-1] - sediment_transport[-2])/spacing

    return -intermittency_factor*time_step*elevation_change/(1 - bed_porosity)

And some parameter values:

In [ ]:
winding_coef = 1
upstream_sediment_transport = sediment_transport[0]

Now let's compute $\partial q_s / \partial x$ only:

In [ ]:
dqdx = exner_equation(sediment_transport,
                      spacing,
                      1,
                      1,
                      0,
                      upstream_sediment_transport,
                      winding_coef)
dqdx *= -1

And plot it:

In [ ]:
fig, ax = plt.subplots()
fig.canvas.header_visible = False
ax.plot(x/1000, dqdx)
ax.set(xlim=(0, extent/1000), #ylim=(-1.5e-9, 0.),
       xlabel='Distance downstream (km)', ylabel=r'$\partial q_s/\partial x$');

## 2. Putting it all together

This section is based on these two blog posts:
 * https://www.andrewjmoodie.com/blog/2017/03-18-2017-building-a-simple-delta-numerical-model-part-v/
 * https://www.andrewjmoodie.com/blog/2017/04-02-2017-building-a-simple-delta-numerical-model-part-vi/

Now all we need to do is put everything into a loop to simulate the evolution of a simple delta over time.

Let's summarize all the parameters required for that:

In [ ]:
# Domain
extent = 1200e3 # m
shape = 400 # cell
spacing = extent/shape # m

# Time
run_time = 500 # yr
time_step = 0.1 # yr
save_step = 2 # yr
intermittency_factor = 0.2

# Initial topography
initial_elevation = 63 # m
initial_slope = 7e-5 # m/m

# Channel and flow
base_level = 0 # m
channel_width = 1100 # m
water_discharge = 10000 # m3/s
fluid_density = 1000 # kg/m3
friction_coef = 0.0047

# Sediment
grain_size = 0.0003 # m
bed_porosity = 0.6
submerged_specific_gravity = 1.65
beta = 0.64

# Solver
winding_coef = 1

And derive some extra parameters:

In [ ]:
n_steps = int(run_time/time_step) + 1
i_save_step = int(save_step/time_step)
time_step_sec = 31557600*time_step # s

Now we can run a big loop to build our delta through time and save the outcome:

In [ ]:
saved_topography = np.empty((n_steps//i_save_step + 1,) + initial_topography.shape)
saved_topography[0] = initial_topography
saved_flow_depth = np.empty((n_steps//i_save_step + 1,) + initial_topography.shape)
topography = initial_topography.copy()

for i in range(n_steps):
    flow_depth = solve_backwater_profile(topography,
                                         spacing,
                                         channel_width,
                                         friction_coef,
                                         water_discharge,
                                         base_level)
    flow_velocity = water_discharge/(flow_depth*channel_width)
    sediment_transport = engelund_hansen_formula(grain_size,
                                                 flow_velocity,
                                                 fluid_density,
                                                 friction_coef,
                                                 submerged_specific_gravity,
                                                 beta)
    elevation_change = exner_equation(sediment_transport,
                                      spacing,
                                      time_step_sec,
                                      intermittency_factor,
                                      bed_porosity,
                                      sediment_transport[0],
                                      winding_coef)
    topography += elevation_change
    if i%i_save_step == 0 and i > 0:
        saved_topography[i//i_save_step] = topography
        saved_flow_depth[i//i_save_step - 1] = flow_depth

saved_flow_depth[-1] = solve_backwater_profile(saved_topography[-1],
                                               spacing,
                                               channel_width,
                                               friction_coef,
                                               water_discharge,
                                               base_level)

And plot the result in a video:

In [ ]:
fig, ax = plt.subplots()
fig.canvas.header_visible = False

line_water, = ax.plot(x/1000, saved_topography[0] + saved_flow_depth[0],
                      c='tab:blue', label='Water surface')
ax.plot(x/1000, saved_topography[0], c='gray')
line_bed, = ax.plot(x/1000, saved_topography[0], c='black', label='Channel bed')
ax.set(xlim=(0, extent/1000), ylim=(-40, 80),
       xlabel='Distance downstream (km)', ylabel='Elevation (m)')
ax.legend()

plt.close()

def update(i):
    line_water.set_ydata(saved_topography[i] + saved_flow_depth[i])
    line_bed.set_ydata(saved_topography[i])

ani = animation.FuncAnimation(fig, update, range(saved_topography.shape[0]), interval=50)
HTML(ani.to_jshtml())